In [10]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup as bs
import pickle
import requests
import time
import lxml
from datetime import datetime, timedelta
from pytz import timezone

import yfinance as yf
import stockstats

import unicodedata
import json
from textblob import TextBlob
from urllib.parse import quote, parse_qs, urlparse, urlencode

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, IsolationForest,VotingClassifier
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.tree import export_graphviz, DecisionTreeClassifier, tree
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report, accuracy_score

# Global Variables

In [2]:
stock_date_start = "2009-12-30"
stock_date_end = "2019-12-31"
target_stock = "AAPL"

# Read Data

In [3]:
stock_without_absolute = pd.read_pickle('./data/stock_without_absolute.pkl')
stock_with_absolute = pd.read_pickle('./data/stock_with_absolute.pkl')

label_abs_1d = pd.read_pickle('./data/label_abs_1d.pkl')
label_abs_7d = pd.read_pickle('./data/label_abs_7d.pkl')
label_abs_30d = pd.read_pickle('./data/label_abs_30d.pkl')

label_value_1d = pd.read_pickle('./data/label_value_1d.pkl')
label_value_7d = pd.read_pickle('./data/label_value_7d.pkl')
label_value_30d = pd.read_pickle('./data/label_value_30d.pkl')


In [1]:
best_1d_model_param = None
best_7d_model_param = None
best_30d_model_param = None

# XGBClassifier

In [ ]:
# stock_with_absolute, predict 1 day trend 
best_parameters = {'gamma': 0.8, 'max_depth': 2, 'min_child_weight': 7, 'n_estimators': 100, "n_jobs": -1}
rfc = XGBClassifier(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_1d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/XGBoost/XGB_1d.pkl','wb'))

In [ ]:
# stock_with_absolute, predict 7 day trend 
best_parameters = {'gamma': 0.8, 'max_depth': 20, 'min_child_weight': 3, 'n_estimators': 100, "n_jobs": -1}
rfc = XGBClassifier(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_7d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/XGBoost/XGB_7d.pkl','wb'))

In [ ]:
# stock_with_absolute, predict 30 day trend 
best_parameters = {'gamma': 0.8, 'max_depth': 20, 'min_child_weight': 3, 'n_estimators': 100}
rfc = XGBClassifier(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_30d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/XGBoost/XGB_30d.pkl','wb'))

# RandomForest

In [ ]:
# stock_with_absolute, predict 1 day trend 
best_parameters = {'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 100}
rfc = RandomForestClassifier(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_1d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/RandomForest/RFC_1d.pkl','wb')) # store model to .pkl

In [ ]:
# stock_with_absolute, predict 7 day trend 
best_parameters = {'max_depth': 20, 'max_features': 'sqrt', 'n_estimators': 1000}
rfc = RandomForestClassifier(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_7d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/RandomForest/RFC_7d.pkl','wb'))

In [ ]:
# stock_with_absolute, predict 30 day trend 
best_parameters = {'max_depth': 20, 'max_features': 'sqrt', 'n_estimators': 100}
rfc = RandomForestClassifier(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_30d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/RandomForest/RFC_30d.pkl','wb'))

# DecisionTree

In [ ]:
# stock_with_absolute, predict 1 day trend 
best_parameters = {'max_depth': 17, 'max_features': 2, 'min_samples_split': 30}
rfc = DecisionTreeClassifier(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_1d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/DecisionTree/DT_1d.pkl','wb'))

In [ ]:
# stock_with_absolute, predict 7 day trend 
best_parameters = {'max_depth': 17, 'max_features': None, 'min_samples_split': 10}
rfc = DecisionTreeClassifier(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_7d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/DecisionTree/DT_7d.pkl','wb'))

In [ ]:
# stock_with_absolute, predict 30 day trend 
best_parameters = {'max_depth': 11, 'max_features': None, 'min_samples_split': 20}
rfc = DecisionTreeClassifier(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_30d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/DecisionTree/DT_30d.pkl','wb'))

# LogisticRegression

In [ ]:
# stock_with_absolute, predict 1 day trend 
best_parameters = {'solver': 'lbfgs'}
rfc = LogisticRegression(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_1d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/LogisticRegression/LR_1d.pkl','wb'))

In [ ]:
# stock_with_absolute, predict 7 day trend 
best_parameters = {'solver': 'newton-cg'}
rfc = LogisticRegression(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_7d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/LogisticRegression/LR_7d.pkl','wb'))

In [ ]:
# stock_with_absolute, predict 30 day trend 
best_parameters = {'solver': 'liblinear'}
rfc = LogisticRegression(**best_parameters)

# train data and label
train_data = stock_with_absolute
label = label_abs_30d

X_train, X_test, y_train, y_test = train_test_split(train_data, label, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rfc.fit(X_train, y_train)

predicted_test = rfc.predict(X_test)
predicted_train = rfc.predict(X_train)
decision_function = rfc.predict_proba(X_test)

acc_train = accuracy_score(y_train, predicted_train)
acc_test = accuracy_score(y_test, predicted_test)
print("The Train Accuracy  %0.3f" % (acc_train))
print("The Test Accuracy   %0.3f" % (acc_test ))
pickle.dump(rfc, open('./backend/AAPL/LogisticRegression/LR_30d.pkl','wb'))

# Load and Process Data

In [ ]:
import pandas as pd
import numpy as np

import bs4 as bs
import pickle
import requests
import time
import lxml
from datetime import datetime, timedelta

import yfinance as yf
import stockstats

import unicodedata
import json
from textblob import TextBlob
from pytz import timezone

In [65]:
from bs4 import BeautifulSoup as bs
def get_news(symbol, time):
    USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
    headers={"user-agent" : USER_AGENT}
    target='AAPL'
    keyword = quote(symbol.encode('utf8'))
    target_param = {
        "tbm": "nws",
        "hl": "en",
        "lr": "lang_en",
        "q": keyword,
        "oq": keyword,
        "dcr": "0",
        "source": "lnt",
        "num": 5,
        "tbs": "cdr:1,cd_min:"+time+",cd_max:"+time,
    }
    url = "https://www.google.com.tw/search?" + urlencode(target_param)
    res = requests.get(url, headers=headers)
    search_list = []
    if res.status_code == 200:
        content = res.content
        soup = bs(content, "html.parser")
        items = soup.findAll("div", {"class": "g"})
        if items:
            for index, item in enumerate(items):
                    # title
                    news_title = item.find("h3", {"class": "r"}).find("a").text
                    # url
                    href = item.find("h3", {"class": "r"}).find("a").get("href")
                    news_link = href


                    # content
                    news_text = item.find("div", {"class": "st"}).text

                    # source
                    news_source = item.find("h3", {"class": "r"}).findNext('div').text.split('-')
                    news_from = news_source[0]
                    time_created = str(news_source[1])

                    # add item into json object
                    search_list.append({
                        "news_title": news_title.replace('...', ''),
                        "news_link": news_link,
                        "news_text": news_text.encode("ascii", "ignore").decode("ascii").replace('...', ''),
                        "news_from": news_from,
                        "time_created": time_created
                    })
    else:
        print('error at '+str(i))
    return search_list, url

In [66]:
stock_date_start = "2020-03-01"

stock_with_absolute = pd.read_pickle('./data/stock_with_absolute.pkl')
label_abs_1d = pd.read_pickle('./data/label_abs_1d.pkl')
X_train, X_test, y_train, y_test = train_test_split(stock_with_absolute, label_abs_1d, test_size=0.1, random_state=42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

stock_raw = yf.download(target_stock, start=stock_date_start)
stock_raw_data = stock_raw
stock_raw_data['high_low_diff'] = (stock_raw_data['High'] - stock_raw_data['Low'])
stock_raw_data['open_close_diff'] = (stock_raw_data['Open'] - stock_raw_data['Close'])
stock_raw_data['high_low_diff_ratio'] = (stock_raw_data['High'] - stock_raw_data['Low']) / stock_raw_data['Close']
stock_raw_data['open_close_diff_ratio'] = (stock_raw_data['Open'] - stock_raw_data['Close']) / stock_raw_data['Close']
stock_stats_data = stockstats.StockDataFrame.retype(stock_raw_data)
stock_stats_data[['change', 'open_delta','close_delta','volume_delta', 'close_-2_r','close_-6_r', 'boll', 'boll_ub', 
                  'boll_lb', 'boll_-1_d', 'boll_ub_-1_d', 'boll_lb_-1_d' , 'kdjk','kdjd','kdjj', 'macd','macds',
                  'macdh', 'rsi_6', 'rsi_12', 'wr_6', 'wr_12', 'cci', 'atr', 'dma', 'vr']]
stock_data = pd.DataFrame(stock_stats_data)
stock_data = stock_data.dropna()
stock_data['boll_k_diff'] = stock_data['boll'] - stock_data['close']

stock_with_absolute = stock_data[['change', 'open_delta','close_delta','volume_delta', 'high_low_diff_ratio', 
                                'open_close_diff_ratio','close_-2_r','close_-6_r','kdjk','kdjd','kdjj', 'macd',
                                'macds', 'macdh', 'rsi_6', 'rsi_12', 'wr_6', 'wr_12', 'cci', 'atr', 'dma', 'vr', 
                                'boll_-1_d','boll_ub_-1_d', 'boll_lb_-1_d', 'boll_k_diff', 'high_low_diff', 
                                'open_close_diff', 'open', 'high', 'low', 'close', 'adj close', 'volume', ]]

print("Process data from: ", stock_with_absolute.index[0], " to ", stock_with_absolute.index[-1])
target = stock_with_absolute.tail(2)

tz = timezone('EST')
EST_Timezone = datetime.now(tz)
if not EST_Timezone.isoweekday() in range(1, 6):
    target_row = target_row.tail(1)
else:
    trading_start = datetime(EST_Timezone.year, EST_Timezone.month, EST_Timezone.day, 9, 30,tzinfo=tz)
    trading_end = datetime(EST_Timezone.year, EST_Timezone.month, EST_Timezone.day, 16, 0, tzinfo=tz)
    if(trading_start <= EST_Timezone <= trading_end):
        target = target.head(1)
    else:
        target = target.tail(1)

news_list, google_url = get_news("AAPL", target.index[0].strftime("%m/%d/%Y"))
num_news = len(news_list)
sentiment = 0
des_sentiment = 0
for news in news_list:
    news_title = news['news_title'].replace('...', '')
    news_des = news['news_text'].encode("ascii", "ignore").decode("ascii").replace('...', '')
    blob = TextBlob(news_title)
    des_blob = TextBlob(news_des)
    sentiment += blob.sentiment.polarity
    des_sentiment += des_blob.sentiment.polarity
target["news_title_score"] = sentiment / num_news
target["news_des_score"] = des_sentiment / num_news

print(target.columns)
SP500_raw = yf.download("^GSPC", start=stock_date_start)
SP500_raw_data = SP500_raw
SP500_raw_data['high_low_diff'] = (SP500_raw['High'] - SP500_raw['Low'])
SP500_raw_data['open_close_diff'] = (SP500_raw['Open'] - SP500_raw['Close'])
SP500_raw_data['high_low_diff_ratio'] = (SP500_raw['High'] - SP500_raw['Low']) / SP500_raw['Close']
SP500_raw_data['open_close_diff_ratio'] = (SP500_raw['Open'] - SP500_raw['Close']) / SP500_raw['Close']
SP500_stats_data = stockstats.StockDataFrame.retype(SP500_raw_data)
SP500_stats_data[['change','close_delta','volume_delta', 'close_-2_r','close_-6_r']]

SP500_stock = pd.DataFrame(SP500_stats_data).add_prefix('sp500_')
target = target.join(SP500_stock.loc[target.index])

gold_raw = yf.download("GLD", start=stock_date_start)
gold_raw_data = gold_raw
# Add high_low_diff, open_close_diff, high_low_diff_ratio, open_close_diff_ratio
gold_raw_data['high_low_diff'] = (gold_raw['High'] - gold_raw['Low'])
gold_raw_data['open_close_diff'] = (gold_raw['Open'] - gold_raw['Close'])
gold_raw_data['high_low_diff_ratio'] = (gold_raw['High'] - gold_raw['Low']) / gold_raw['Close']
gold_raw_data['open_close_diff_ratio'] = (gold_raw['Open'] - gold_raw['Close']) / gold_raw['Close']

# Add financial indicators
gold_stats_data = stockstats.StockDataFrame.retype(gold_raw_data)
gold_stats_data[['change','close_delta','volume_delta', 'close_-2_r','close_-6_r']]

gold_stock = pd.DataFrame(gold_stats_data).add_prefix('gold_')
target = target.join(gold_stock.loc[target.index])

# 5 year bonds
y5bond_raw = yf.download("^FVX", start=stock_date_start)
y5bond_raw_data = y5bond_raw
y5bond_raw_data['high_low_diff'] = (y5bond_raw['High'] - y5bond_raw['Low'])
y5bond_raw_data['open_close_diff'] = (y5bond_raw['Open'] - y5bond_raw['Close'])
y5bond_raw_data['high_low_diff_ratio'] = (y5bond_raw['High'] - y5bond_raw['Low']) / y5bond_raw['Close']
y5bond_raw_data['open_close_diff_ratio'] = (y5bond_raw['Open'] - y5bond_raw['Close']) / y5bond_raw['Close']
y5bond_stats_data = stockstats.StockDataFrame.retype(y5bond_raw_data)
y5bond_stats_data[['change','close_delta', 'close_-2_r','close_-6_r']] ### no volume

y5bond_stock = pd.DataFrame(y5bond_stats_data).drop(columns='volume').add_prefix('y5bond_')

# 10 year bonds
y10bond_raw = yf.download("^TNX", start=stock_date_start)
y10bond_raw_data = y10bond_raw
y10bond_raw_data['high_low_diff'] = (y10bond_raw['High'] - y10bond_raw['Low'])
y10bond_raw_data['open_close_diff'] = (y10bond_raw['Open'] - y10bond_raw['Close'])
y10bond_raw_data['high_low_diff_ratio'] = (y10bond_raw['High'] - y10bond_raw['Low']) / y10bond_raw['Close']
y10bond_raw_data['open_close_diff_ratio'] = (y10bond_raw['Open'] - y10bond_raw['Close']) / y10bond_raw['Close']
y10bond_stats_data = stockstats.StockDataFrame.retype(y10bond_raw_data)
y10bond_stats_data[['change','close_delta', 'close_-2_r','close_-6_r']] ### no volume

y10bond_stock = pd.DataFrame(y10bond_stats_data).drop(columns='volume').add_prefix('y10bond_')
target = target.join(y10bond_stock.loc[target.index])
target = target.join(y5bond_stock.loc[target.index])
return_dict = {}
return_dict["Data"] = target.reset_index(drop=True).transpose().to_dict()
target = scaler.transform(target)
return_dict["news"] = news_list
return_dict["news_url"] = google_url
return_dict["LGR"] = [int(pickle.load(open('./backend/AAPL/LogisticRegression/LR_1d.pkl','rb')).predict(target)[0]),
                      int(pickle.load(open('./backend/AAPL/LogisticRegression/LR_7d.pkl','rb')).predict(target)[0]),
                      int(pickle.load(open('./backend/AAPL/LogisticRegression/LR_30d.pkl','rb')).predict(target)[0])]
# return_dict["DT"] = [pickle.load(open('./backend/AAPL/DecisionTree/DT_1d.pkl','rb')).predict(target)[0],
#                       pickle.load(open('./backend/AAPL/DecisionTree/DT_7d.pkl','rb')).predict(target)[0],
#                       pickle.load(open('./backend/AAPL/DecisionTree/DT_30d.pkl','rb')).predict(target)[0]]
# return_dict["RF"] = [pickle.load(open('./backend/AAPL/RandomForest/RFC_1d.pkl','rb')).predict(target)[0],
#                       pickle.load(open('./backend/AAPL/RandomForest/RFC_7d.pkl','rb')).predict(target)[0],
#                       pickle.load(open('./backend/AAPL/RandomForest/RFC_30d.pkl','rb')).predict(target)[0]]
# return_dict["XGB"] = [pickle.load(open('./backend/AAPL/XGBoost/XGB_1d.pkl','rb')).predict(target)[0],
#                       pickle.load(open('./backend/AAPL/XGBoost/XGB_7d.pkl','rb')).predict(target)[0],
#                       pickle.load(open('./backend/AAPL/XGBoost/XGB_30d.pkl','rb')).predict(target)[0]]
return_dict

[*********************100%***********************]  1 of 1 completed

/Users/chinkashiwakin/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values



Process data from:  2020-03-10 00:00:00  to  2020-04-27 00:00:00
Index(['change', 'open_delta', 'close_delta', 'volume_delta',
       'high_low_diff_ratio', 'open_close_diff_ratio', 'close_-2_r',
       'close_-6_r', 'kdjk', 'kdjd', 'kdjj', 'macd', 'macds', 'macdh', 'rsi_6',
       'rsi_12', 'wr_6', 'wr_12', 'cci', 'atr', 'dma', 'vr', 'boll_-1_d',
       'boll_ub_-1_d', 'boll_lb_-1_d', 'boll_k_diff', 'high_low_diff',
       'open_close_diff', 'open', 'high', 'low', 'close', 'adj close',
       'volume', 'news_title_score', 'news_des_score'],
      dtype='object')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'Data': {0: {'change': 2.8869586869241237,
   'open_delta': 1.33001708984375,
   'close_delta': 7.94000244140625,
   'volume_delta': 336700.0,
   'high_low_diff_ratio': 0.021239035020314673,
   'open_close_diff_ratio': -0.020390815241123593,
   'close_-2_r': 2.4882270790721472,
   'close_-6_r': -1.2975692172814468,
   'kdjk': 60.68413333886924,
   'kdjd': 63.54806129349879,
   'kdjj': 54.95627742961014,
   'macd': 5.483108707113217,
   'macds': 4.440208391327328,
   'macdh': 1.0429003157858894,
   'rsi_6': 63.057597243750884,
   'rsi_12': 58.05505729000778,
   'wr_6': 18.494458058221586,
   'wr_12': 19.541084108602867,
   'cci': 57.9435015699361,
   'atr': 11.044472937388504,
   'dma': 13.961997085962537,
   'vr': 70.94092501727684,
   'boll_-1_d': 1.226499938964821,
   'boll_ub_-1_d': 2.0172420923347545,
   'boll_lb_-1_d': 0.4357577855948307,
   'boll_k_diff': -16.28650360107423,
   'high_low_diff': 6.010009765625,
   'open_close_diff': -5.769989013671875,
   'open': 277.200012207031

In [67]:
print(return_dict['Data'].keys())

dict_keys([0])


In [68]:
from flask import Flask, request, jsonify, render_template, Response
json.dumps(return_dict)

'{"Data": {"0": {"change": 2.8869586869241237, "open_delta": 1.33001708984375, "close_delta": 7.94000244140625, "volume_delta": 336700.0, "high_low_diff_ratio": 0.021239035020314673, "open_close_diff_ratio": -0.020390815241123593, "close_-2_r": 2.4882270790721472, "close_-6_r": -1.2975692172814468, "kdjk": 60.68413333886924, "kdjd": 63.54806129349879, "kdjj": 54.95627742961014, "macd": 5.483108707113217, "macds": 4.440208391327328, "macdh": 1.0429003157858894, "rsi_6": 63.057597243750884, "rsi_12": 58.05505729000778, "wr_6": 18.494458058221586, "wr_12": 19.541084108602867, "cci": 57.9435015699361, "atr": 11.044472937388504, "dma": 13.961997085962537, "vr": 70.94092501727684, "boll_-1_d": 1.226499938964821, "boll_ub_-1_d": 2.0172420923347545, "boll_lb_-1_d": 0.4357577855948307, "boll_k_diff": -16.28650360107423, "high_low_diff": 6.010009765625, "open_close_diff": -5.769989013671875, "open": 277.20001220703125, "high": 283.010009765625, "low": 277.0, "close": 282.9700012207031, "adj clos